In [61]:
import pickle

In [30]:
import numpy as np
import pandas as pd

In [ ]:
import nltk
from nltk.corpus import stopwords

# Baixar recursos do NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

# Definindo as nossas stopwords
stop_words = set(stopwords.words('portuguese'))

# Carregar o modelo de lematização
import spacy
lemmatizer = spacy.load("pt_core_news_sm")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rodri\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rodri\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\rodri\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [34]:
# Carregar datasets
df_train = pd.read_parquet(R'C:\Users\rodri\Desktop\datathon-fiap-final\train\train-all-parts.parquet', engine='pyarrow')
df_items = pd.read_parquet(R'C:\Users\rodri\Desktop\datathon-fiap-final\itens\itens-all-parts.parquet', engine='pyarrow')
df_validation = pd.read_parquet(R'C:\Users\rodri\Desktop\datathon-fiap-final\validation.parquet', engine='pyarrow')

In [35]:
# Tipagem das colunas para otimização de memória
df_train = df_train.astype({
    'userId': 'string',
    'userType': 'category',
    'historySize': 'uint8',
    'history': 'string',
    'timestampHistory': 'string',
    'numberOfClicksHistory': 'string',
    'timeOnPageHistory': 'string',
    'scrollPercentageHistory': 'string',
    'pageVisitsCountHistory': 'string',
})

df_items = df_items.astype({
    'page': 'string',
    'url': 'string',
    'issued': 'string',
    'modified': 'string',
    'title': 'string',
    'body': 'string',
    'caption': 'string'
})

df_validation = df_validation.astype({
    'userId': 'string',
    'userType': 'category',
    'history': 'string',
    'timestampHistory': 'string'
})

In [36]:
from IPython.display import display, HTML

def display_dataframe_info(df: pd.DataFrame, title: str) -> None:
    display(HTML(f"<h3 style='color:yellow; font-weight:bold; margin:0;'>{title}</h3>"))
    display(HTML(f"<p style='margin:0; padding-left:20px;'>Shape: {df.shape}</br>Uso de memória: {df.memory_usage(deep=True).sum() / (1024**2)} MB</br>Columns: {list(df.columns)}</p>"))

display_dataframe_info(df_train, "Informações do DataFrame de Treino")
display_dataframe_info(df_items, "Informações do DataFrame de Itens")
display_dataframe_info(df_validation, "Informações do DataFrame de Validação")

In [37]:
# Função para converter colunas em listas
def convert_to_list(df: pd.DataFrame, column_name: str, dtype: str = str) -> pd.Series:
    if column_name not in df.columns:
        raise ValueError(f"Column '{column_name}' not found in DataFrame")
    
    def safe_convert(value):
        if isinstance(value, str):
            return list(map(dtype, value.split(', ')))
        elif pd.isna(value):
            return []
        else:
            return [dtype(value)]
    
    return df[column_name].apply(safe_convert)

In [38]:
# Aplicar conversão para listas
df_train['history'] = convert_to_list(df_train, 'history', dtype=str)
df_train['timestampHistory'] = convert_to_list(df_train, 'timestampHistory', dtype=np.int64)
df_train['numberOfClicksHistory'] = convert_to_list(df_train, 'numberOfClicksHistory', dtype=np.uint16)
df_train['timeOnPageHistory'] = convert_to_list(df_train, 'timeOnPageHistory', dtype=np.int32)
df_train['scrollPercentageHistory'] = convert_to_list(df_train, 'scrollPercentageHistory', dtype=np.float32)
df_train['pageVisitsCountHistory'] = convert_to_list(df_train, 'pageVisitsCountHistory', dtype=np.uint16)

# Criar novas features
df_train['avg_time_on_page'] = df_train['timeOnPageHistory'].apply(lambda x: np.mean(x) if x else 0)
df_train['total_clicks'] = df_train['numberOfClicksHistory'].apply(np.sum)
df_train['avg_scroll_percentage'] = df_train['scrollPercentageHistory'].apply(lambda x: np.mean(x) if x else 0)

df_train['timestampHistory'] = df_train['timestampHistory'].apply(lambda x: pd.to_datetime(x, errors='coerce'))
df_train['recency_days'] = (pd.Timestamp.now() - df_train['timestampHistory'].apply(max)).dt.days

df_train = df_train.dropna(subset=['recency_days']).reset_index(drop=True)

In [39]:
def remove_stopwords(text: str) -> str:
    # Converte o texto para minúsculo
    text = text.lower()

    # Processa o texto com SpaCy (tokenização + POS tagging + etc.)
    doc = lemmatizer(text)

    processed_tokens = []
    for token in doc:
        # Checa se não é pontuação, não é espaço, não é stopword 
        # (você pode usar tanto a lista 'stop_words' quanto token.is_stop)
        if not token.is_punct and not token.is_space and token.text not in stop_words:
            # Usa a forma lematizada do token
            processed_tokens.append(token.lemma_)

    # Junta os tokens lematizados em uma string novamente
    return ' '.join(processed_tokens)

In [ ]:
# Execute esse código caso o treinamento esteja muito lento.

# def remove_stopwords(text: str) -> str:
#     words = word_tokenize(str(text).lower())
#     words = [word for word in words if word.isalnum() and word not in stop_words]

#     return ' '.join(words)

In [41]:
df_items_copy = df_items.copy()

# Processamento dos itens
df_items['issued'] = pd.to_datetime(df_items['issued'], errors='coerce')
df_items['modified'] = pd.to_datetime(df_items['modified'], errors='coerce')
df_items['recency_days'] = (pd.to_datetime(df_items['modified']).max() - df_items['modified']).dt.days


df_items['title_clean'] = df_items['title'].apply(remove_stopwords)
df_items['body_clean'] = df_items['body'].apply(remove_stopwords)
df_items['content_clean'] = (df_items['title_clean'] + ' ' + df_items['body_clean']).str.strip()
df_items = df_items.drop_duplicates(subset=['page']).reset_index(drop=True)

In [42]:
# Explodir histórico de leitura
df_user_news = df_train.explode('history')
df_user_news.rename(columns={'history': 'page'}, inplace=True)

# União do dataframe de treino com items
df_train_final = df_user_news.merge(df_items, on='page', how='left')

In [43]:
# Criar vetores TF-IDF para o conteúdo das notícias
vectorizer = TfidfVectorizer(
    stop_words=list(stop_words),
    max_features=2000,
    max_df=0.95,
    min_df=0.05,
    ngram_range=(1, 2)
)
tfidf_matrix = vectorizer.fit_transform(df_items['content_clean'])

In [44]:
# Calcular a similaridade em lotes para evitar estouro de memória
def batch_cosine_similarity(matrix, batch_size=1000):
    n = matrix.shape[0]
    similarity_dict = {}

    for start in range(0, n, batch_size):
        end = min(start + batch_size, n)
        batch_sim = cosine_similarity(matrix[start:end], matrix)

        for i, idx in enumerate(range(start, end)):
            similar_indices = batch_sim[i].argsort()[-6:-1][::-1]  # Pegamos as 5 mais similares (excluindo ela mesma)
            similarity_dict[df_items.iloc[idx]['page']] = df_items.iloc[similar_indices]['page'].tolist()

    return similarity_dict

In [45]:
similar_noticias = batch_cosine_similarity(tfidf_matrix)

In [ ]:
# df_user_news já possui colunas: ['userId', 'page', 'timestampHistory', 'total_clicks', ...]
# Precisamos sumarizar ou criar algum score de popularidade por notícia

popularity = (
    df_user_news
    .groupby('page', as_index=False)
    .agg({
        'total_clicks': 'sum',
        'avg_scroll_percentage': 'mean',
    })
)

# Exemplo de score de popularidade
popularity['popularity_score'] = (
    popularity['total_clicks'] * 0.3 +
    popularity['avg_scroll_percentage'] * 0.5
)

In [49]:
df_items = df_items.merge(popularity[['page', 'popularity_score']], on='page', how='left')
df_items['popularity_score'] = df_items['popularity_score'].fillna(0)

In [ ]:
def recomendar_noticias(
        user_id: str, df_train_final: pd.DataFrame, df_items: pd.DataFrame, similar_noticias: dict, top_n: int = 5) -> pd.DataFrame:
#     """
#     Função para recomendar notícias personalizadas para um usuário.
    
#     Se o usuário tiver histórico suficiente, usa um modelo treinado para prever quais notícias ele consumirá.
#     Se for um usuário novo ou anônimo (cold-start), recomenda notícias populares recentes.
    
#     Args:
#         user_id (str): ID do usuário.
#         df_train_final (DataFrame): Dados de interação dos usuários após processo dos de pré-processamento.
#         df_items (DataFrame): Dados das notícias.
#         top_n (int): Número de recomendações a retornar.

#     Returns:
#         DataFrame: Notícias recomendadas com score ordenado.
#     """

    # Garantir que 'popularity_score' e 'recency_days' existem
    if 'popularity_score' not in df_items.columns:
        print("⚠️ Atenção: 'popularity_score' não encontrado! Substituindo por 0.")
        df_items['popularity_score'] = 0

    if 'recency_days' not in df_items.columns:
        print("⚠️ Atenção: 'recency_days' não encontrado! Substituindo por 0.")
        df_items['recency_days'] = 0

    # 🔹 Verificar se o usuário já tem histórico de leitura
    user_history_df = df_train_final[df_train_final['userId'] == user_id]

    if user_history_df.empty:
        # 🚀 Cold-Start: Recomendar notícias populares
        print(f"⚠️ Usuário {user_id} sem histórico, usando recomendação por popularidade.")

        df_items['engagement_score'] = (
            df_items['popularity_score'] * 0.8 +
            df_items['recency_days'].apply(lambda x: -0.2 * x if pd.notna(x) else 0)
        )
        return df_items.sort_values('engagement_score', ascending=False).head(top_n)

    else:
        # 🚀 Usuário com histórico: Recomendação baseada no modelo
        print(f"✅ Usuário {user_id} encontrado! Usando modelo de Machine Learning.")
        noticias_lidas = user_history_df['page'].unique().tolist()

        noticias_similares = []
        for noticia in noticias_lidas:
            if noticia in similar_noticias:
                noticias_similares.extend(similar_noticias[noticia])

        # Verificar se encontramos notícias similares
        if not noticias_similares:
            print("⚠️ Nenhuma notícia similar encontrada! Retornando notícias populares.")
            return df_items.sort_values('popularity_score', ascending=False).head(top_n)

        df_candidate_news = df_items[df_items['page'].isin(noticias_similares) & ~df_items['page'].isin(noticias_lidas)]

        if df_candidate_news.empty:
            print("⚠️ Nenhuma notícia disponível para recomendação. Retornando populares.")
            return df_items.sort_values('popularity_score', ascending=False).head(top_n)

        df_candidate_news['candidate_score'] = (
            df_candidate_news['popularity_score'] * 0.5 +
            df_candidate_news['recency_days'].apply(lambda x: -0.2 * x if pd.notna(x) else 0)
        )

        return df_candidate_news.sort_values('candidate_score', ascending=False).head(top_n)


In [52]:
# Precisamos do df_train_final ou df_user_news contendo userId + page
df_train_final = df_user_news.merge(df_items, on='page', how='left')

In [53]:
pd.set_option('display.max_columns', None)  # Exibe todas as colunas sem truncamento
pd.set_option('display.max_colwidth', None)  # Exibe todo o conteúdo das células

In [64]:
# Exemplo de chamada
recomendacoes = recomendar_noticias('54462d26984ee3cb49263b3e77c4abe4d4e13023dbbf2d683e2e5c9e114004c1', df_train_final, df_items, similar_noticias)

C:\Users\rodri\AppData\Local\Temp\ipykernel_11476\2824676940.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_candidate_news['candidate_score'] = (


In [65]:
display(recomendacoes[['page', 'title', 'url', 'recency_days']])

,page,title,url,recency_days
60121,1f32787b-de2b-49be-8c20-ddaeae34cc22,Filha é presa por golpe estimado em R$ 725 milhões contra a mãe; quadros renomados roubados foram recuperados,http://g1.globo.com/rj/rio-de-janeiro/noticia/2022/08/10/policia-tenta-prender-pessoas-que-extorquiram-milhoes-de-idosa-no-rio.ghtml,323
126452,5af379e6-1bd1-4cf8-a23c-03266fb77b2c,Casa abandonada em Higienópolis: Entenda o caso da mulher que vive em mansão de SP,http://g1.globo.com/sp/sao-paulo/noticia/2022/07/20/casa-abandonada-em-higienopolis-entenda-o-caso-da-mulher-que-vive-em-mansao-de-sp.ghtml,343
52787,7c076eda-5b41-4b0f-815f-0f6400148b0c,"Câmera registra momento em que apoiador de Bolsonaro invade festa e mata guarda municipal que era tesoureiro do PT, em Foz do Iguaçu",http://g1.globo.com/pr/oeste-sudoeste/noticia/2022/07/10/camera-registra-momento-em-que-atirador-invade-festa-e-mata-guarda-municipal-que-era-tesoureiro-do-pt-em-foz-do-iguacu.ghtml,354
4191,ecc37a22-b730-4e3a-bc87-c3ba3403acbc,PM suspeito de atirar e matar campeão mundial de jiu-jítsu é preso após decisão da Justiça,http://g1.globo.com/sp/sao-paulo/noticia/2022/08/07/justica-determina-prisao-de-pm-suspeito-de-atirar-em-campeao-mundial-de-jiu-jitsu.ghtml,325
120168,8d477e04-3bab-4ad9-8fe3-799059238a9c,"Quem é Giovanni Quintella, anestesista preso em flagrante por estuprar grávida no parto; ele atuou em pelo menos 10 hospitais",http://g1.globo.com/rj/rio-de-janeiro/noticia/2022/07/11/quem-e-giovanni-quintella-anestesista-preso-em-flagrante-por-estuprar-gravida-durante-o-parto-ele-atuou-em-pelo-menos-10-hospitais.ghtml,350


In [63]:
# Caminho para salvar
pickle_path = "./modelo_recomendacao.pkl"

# Criando um dicionário com os três objetos essenciais
modelo_recomendacao = {
    "df_train_final": df_train_final,
    "df_items": df_items,
    "similar_noticias": similar_noticias
}

# Salvando o modelo
with open(pickle_path, 'wb') as f:
    pickle.dump(modelo_recomendacao, f)

print(f"✅ Modelo salvo com sucesso em {pickle_path}!")

✅ Modelo salvo com sucesso em ./modelo_recomendacao.pkl!
